In [14]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

In [15]:
sp_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/sp_power.h5'
sm_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/sm_power.h5'
sm=openmc.Summary(sm_file)
sp = openmc.StatePoint(sp_file)
sp.link_with_summary(sm)
model=openmc.model.Model
model_file='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera/model_2A_72_families_power.xml'
model=openmc.model.Model.from_model_xml(model_file)

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=13.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=14.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=15.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=6013.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=6014.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=6015.
  warn(msg, IDWarn

In [16]:
sp.tallies
for index, tally in enumerate(sp.tallies):
    t=sp.tallies[tally]
def power_cell(cell,x,y):    
    cell_index = 99
    cell_id=99
    i=0
    for all_cell in model.geometry.get_all_material_cells().values():
        i=i+1
        if all_cell.name==cell+'_'+str(x)+'_'+str(y):
            cell_index=all_cell.id
            cell_id=i-1
    tally_data_reshaped =t.get_reshaped_data()
    power=tally_data_reshaped[cell_id]
    if x==8 or y==8:
        return(2*power[0][0])
    else : 
        return(power[0][0])

In [17]:
power_cell('fuel',8,0)

1319793.4134345332

In [18]:
GT=[(5,2),(8,2),(3,3),(2,5),(5,5),(8,5),(2,8),(5,8),(8,8)]
sum_power=0
N=0
for x in range(9):
    for y in range(9):
        if (x,y) not in GT:
            N = N+1
            name='fuel_'+str(x)+'_'+str(y)
            sum_power =sum_power + power_cell('fuel',x,y)

avg_power=sum_power/N


In [19]:
def nappe(cell,x,y):
    A=power_cell(cell,x,y)/avg_power
    return(A)

In [22]:
nappe('fuel',0,0)

1.075003915609741

### Mapping geom

In [30]:
GT=[(5,2),(8,2),(3,3),(2,5),(5,5),(8,5),(2,8),(5,8)]
IT=[(8,8)]

GT_left=[]
GT_full=[]
for X in GT:
    GT_left.append((X[0],X[1]))
    GT_left.append((X[0],16-X[1]))
for X in GT_left:
    GT_full.append((X[0],X[1]))
    GT_full.append((16-X[0],X[1]))    
GT_full = list(set(GT_full))

### Mapping benchmark

In [28]:
A2=[(0,0,0.9476),(1,0,0.9418),(2,0,0.9458),(3,0,0.9551),(4,0,0.9645),(5,0,0.9739),(6,0,0.9714),(7,0,0.9721),(8,0,0.9764),(1,1,0.9389),(2,1,0.9483 ),(3,1,0.9649 ),(4,1,0.9837),(5,1,1.0115),(6,1,0.9880),(7,1,0.9880),(8,1,1.0122),(2,2,0.9736),(3,2,1.0180),(4,2,1.0360),(6,2,1.0281),(7,2,1.0266),(4,3,1.0516),(5,3,1.0458),(6,3,1.0089),(7,3,1.0053),(4,0,0.9645),(5,0,0.9739),(6,0,0.9714),(7,0,0.9721),(8,0,0.9764),(1,1,0.9389),(2,1,0.9483 ),(3,1,0.9649 ),(4,1,0.9837),(5,1,1.0115),(6,1,0.9880),(7,1,0.9880),(8,1,1.0122),(2,2,0.9736),(3,2,1.0180),(4,2,1.0360),(6,2,1.0281),(7,2,1.0266),(4,3,1.0516),(5,3,1.0458),(6,3,1.0089),(7,3,1.0053),(4,0,0.9645),(8,3,0.9645),(4,4, 1.0313 ),(5,4,1.0451),(6,4,1.0118),(7,4,1.0089),(8,4,1.0353),(6,5,1.0386),(7,5,1.0368),(6,6,1.0104),(7,6,1.0093 ),(8,6,1.0371),(7,7,1.0089),(8,7,1.0364)]

In [49]:
rd=[]
for pos in A2:
    if (pos[0],pos[1]) not in GT_full and (pos[0],pos[1]) not in IT:
        result=nappe('fuel',pos[0],pos[1])
        bench=pos[2]
        rd.append((pos[0],pos[1],100*abs(result-bench)/bench))
rd_max=0
x=99
y=99
for A in rd:
    if A[2]>rd_max:
        rd_max=A[2]
        x=A[0]
        y=A[1]
print(x,y,rd_max)   

0 0 13.444904559913578
